In [1]:
import pandas as pd
from prophet import Prophet
import mlflow
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

c:\Users\nico_\anaconda3\envs\ml-flow\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(mlflow.prophet.get_default_conda_env())
print(mlflow.prophet.get_default_pip_requirements())

{'name': 'mlflow-env', 'channels': ['conda-forge'], 'dependencies': ['python=3.11.5', 'pip<=23.3.1', {'pip': ['mlflow==2.8.1', 'prophet==1.1.5']}]}
['prophet==1.1.5']


In [3]:
class ProphetModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model
        super().__init__()

    def predict(self, context, model_input):
        future = self.model.make_future_dataframe(periods=model_input['periods'][0])
        return self.model.predict(future)

In [7]:
df = pd.read_csv(r'C:\Users\nico_\OneDrive\Portfolio\Stock Level Predictions\data\sales.csv', encoding='latin1')

# Assuming 'Order Date' is your date column and 'Order Quantity' is what you want to predict
df = df[['date', 'qty', 'prod_code']]
df.columns = ['ds', 'y', 'prod_code']

# Get list of unique products
products = df['prod_code'].unique()

In [8]:
df.head()

,ds,y,prod_code
0,2017-03-09,5,Go-Wo-NMDVGP
1,2015-01-13,5,Go-Wo-NMDVGP
2,2015-01-28,5,Go-Wo-NMDVGP
3,2015-01-21,5,Go-Wo-NMDVGP
4,2015-01-24,5,Go-Wo-NMDVGP


In [9]:
products

array(['Go-Wo-NMDVGP', 'Fa-In-OMNLV', 'Go-Sh-UAGTSSR', 'Ou-Tr-GIUBGT',
       'Go-Gi-NMDWWB', 'Fo-Ca-NWTS', 'Ou-El-TPVGB', 'Ou-El-TPVHNPG',
       'Ou-Go-HWHGG', 'Go-Gi-aYGBAMS', 'Ou-Tr-GIWMO3CAG', 'Ou-Go-C8SB',
       'Ou-Go-LWESGP', 'Fo-El-UAWIPVS', 'Ou-Tr-GIIGT', 'Ou-El-BeSDNSD',
       'Fo-El-UAWIS', 'Ou-Go-GIWIGG', 'Go-Gi-TBTDJ', 'Fo-El-UAKMS',
       'Fi-Ba-aMFMTFSC', 'Ou-Go-HWSFGG', 'Fi-Te-NMC2S', 'Ou-El-BeSDNC',
       'Ou-Go-CRCB', 'Fo-Fi-UAHSMDB', 'Ou-El-BeSDNT', 'Fo-Bo-UAWMGSRS',
       'Ou-Go-GIWMOGG', 'Go-Sh-CMPATT', 'Fa-Hu-EAHS', 'Fo-El-UAMCESS',
       'Ou-Go-TFW2XH', 'Ou-Go-HMHGG', 'Fi-Ho-NDCS6P', 'Fi-Ba-aKFMFSC',
       'Ou-El-TPVHNGB', 'Fo-As-NMFT5TTS', 'Fi-So-ETM2', 'Ou-Ki-PFHRM',
       'Fo-Bo-BWG6RS', 'Ou-Wo-TWSIP', 'Fo-Bo-NWF5TFP4TS', 'Ou-Go-LWMPGS',
       'Ou-Go-BBMPC', 'Ou-Me-MWGSHS', 'Fa-Hu-TNFWRB', 'Ou-Go-BBBH',
       'Ou-Ac-TGSFGPG', 'Fi-Ho-NWLVT', 'Ou-Ac-TGTLPG', 'Go-Gi-aMGBCAT',
       'Ou-Ac-TGNEPPG', 'Ou-Ac-TGTVPG', 'Ou-Ac-TGSLCPG', 'Ou-Ac-TGPSPG',
    

In [10]:
for product in products:
    # Filter data for one product
    df_product = df[df['prod_code'] == product]
    
    # Initialize and fit the model
    m = Prophet()
    m.fit(df_product)
    
    # Make future dataframe for prediction
    future = m.make_future_dataframe(periods=365)  # Predict for next 365 days
    
    # Predict
    forecast = m.predict(future)
    # Log model
    with mlflow.start_run():
        mlflow.log_param("ProductCode", product)
        mlflow.pyfunc.log_model("model", python_model=ProphetModel(m))
        # Log future dataframe as an artifact
        future.to_csv("future.csv", index=False)
        mlflow.log_artifact("future.csv")


20:04:24 - cmdstanpy - INFO - Chain [1] start processing
20:04:24 - cmdstanpy - INFO - Chain [1] done processing
20:04:28 - cmdstanpy - INFO - Chain [1] start processing
20:04:28 - cmdstanpy - INFO - Chain [1] done processing
20:04:32 - cmdstanpy - INFO - Chain [1] start processing
20:04:32 - cmdstanpy - INFO - Chain [1] done processing
20:04:35 - cmdstanpy - INFO - Chain [1] start processing
20:04:35 - cmdstanpy - INFO - Chain [1] done processing
20:04:38 - cmdstanpy - INFO - Chain [1] start processing
20:04:38 - cmdstanpy - INFO - Chain [1] done processing
20:04:41 - cmdstanpy - INFO - Chain [1] start processing
20:04:42 - cmdstanpy - INFO - Chain [1] done processing
20:04:45 - cmdstanpy - INFO - Chain [1] start processing
20:04:45 - cmdstanpy - INFO - Chain [1] done processing
20:04:48 - cmdstanpy - INFO - Chain [1] start processing
20:04:48 - cmdstanpy - INFO - Chain [1] done processing
20:04:51 - cmdstanpy - INFO - Chain [1] start processing
20:04:51 - cmdstanpy - INFO - Chain [1]

In [9]:
# Get the artifact location of a run by run id
# artifact_uri = mlflow.get_artifact_uri('dbe54bcc19e04a07bb90c78b592fbb92')
# Load the future dataframe from the CSV file
future = pd.read_csv(r"C:\Users\nico_\OneDrive\Portfolio\Stock Level Predictions\mlruns\397427519765463275\0afd40de0ef441bab9b4e554ca2f9d2a\artifacts\future.csv", encoding='latin1')
# Plot the future dataframe
plt.plot(future["ds"], future["yhat"])
plt.xlabel("Date")
plt.ylabel("Forecast")
plt.title("Future Forecast for Product Code " + product)
plt.show()

KeyError: 'yhat'